In [1]:
import pyarrow.parquet as pq
import pandas as pd

files = {
    "COAD": "aggregated_methylation_data/final_methylation_data_TCGA-COAD.parquet",
    "READ": "aggregated_methylation_data/final_methylation_data_TCGA-READ.parquet",
    "LUAD": "aggregated_methylation_data/final_methylation_data_TCGA-LUAD.parquet",
}

batch_size = 1000
df_meth_COAD_combined = None
df_meth_READ_combined = None
df_meth_LUAD_combined = None

for cancer_type, file_path in files.items():
    chunks = []
    reader = pq.ParquetFile(file_path, thrift_string_size_limit=2147483647)
    for batch in reader.iter_batches(batch_size=batch_size):
        df_chunk = batch.to_pandas()
        chunks.append(df_chunk)
    
    combined_df = pd.concat(chunks, ignore_index=True)

    if cancer_type == "COAD":
        df_meth_COAD_combined = combined_df
    elif cancer_type == "READ":
        df_meth_READ_combined = combined_df
    elif cancer_type == "LUAD":
        df_meth_LUAD_combined = combined_df

In [2]:
print(df_meth_COAD_combined.head())
print(df_meth_READ_combined.head()) 

   cg00000108  cg00000109  cg00000165  cg00000236  cg00000289  cg00000292  \
0    0.953630    0.907077    0.144973    0.912979    0.813640    0.765790   
1    0.972363    0.882373    0.580443    0.923796         NaN    0.746361   
2    0.956964    0.894452    0.306804    0.888815         NaN    0.698911   
3    0.966731    0.929120    0.471386    0.947110         NaN    0.761455   
4    0.969658    0.920940    0.152327    0.933957    0.793562    0.633932   

   cg00000321  cg00000363  cg00000622  cg00000658  ...  rs7746156  rs798149  \
0    0.387424    0.254435    0.016053    0.908794  ...        NaN       NaN   
1    0.419409    0.519763    0.014923    0.910423  ...        NaN       NaN   
2    0.268164    0.180543    0.019389    0.884833  ...        NaN       NaN   
3    0.462856    0.443001    0.020803    0.887060  ...        NaN       NaN   
4    0.372154    0.155002    0.015081    0.877770  ...        NaN       NaN   

   rs845016  rs877309  rs9292570  rs9363764  rs939290  rs95129

In [ ]:
import pandas as pd
import os

def get_top_mad_cpgs(df, num_mad_cpgs=100000):
    df = df.dropna(axis=1, how='any')
    print(f"Number of CpG sites remaining after dropping rows with NaN: {df.shape[1]}")

    print(df.head())

    mad_cpgs = df.apply(lambda x: (abs(x - x.median())).median(), axis=0).sort_values(ascending=False)
    
    print(mad_cpgs.head())
    
    top_mad_cpgs = mad_cpgs.iloc[:num_mad_cpgs].index
    methyl_subset_df = df.loc[:, top_mad_cpgs]
    
    return methyl_subset_df, top_mad_cpgs

datasets = {
    "COAD": df_meth_COAD_combined,
    "READ": df_meth_READ_combined,
    "LUAD": df_meth_LUAD_combined
}

# Ensure all datasets have exactly the same CpG sites
reference_cpg_sites = set(datasets["COAD"].columns)
for cancer_type, df in datasets.items():
    if set(df.columns) != reference_cpg_sites:
        raise ValueError(f"ERROR: {cancer_type} CpG sites do NOT match exactly. Fix the issue before proceeding.")

# Combine all datasets along rows (concatenating samples)
combined_df = pd.concat(datasets.values(), axis=0)

# Get top most variable CpG sites
filtered_df, top_mad_cpgs = get_top_mad_cpgs(combined_df)

output_dir = 'filtered_methylation_data'
os.makedirs(output_dir, exist_ok=True)
cpg_sites_file = os.path.join(output_dir, 'top_100k_most_variable_cpg_sites.parquet')
pd.DataFrame(top_mad_cpgs, columns=['CpG_Site']).to_parquet(cpg_sites_file, index=False)

print(f"✔ Saved top 100,000 CpG sites: {cpg_sites_file}")

output_files = {}
for cancer_type, df in datasets.items():
    subset_df = df.loc[:, top_mad_cpgs]
    output_file = os.path.join(output_dir, f'{cancer_type.lower()}_top100kMAD_cpg.parquet')
    subset_df.to_parquet(output_file)
    output_files[cancer_type] = output_file
    print(f"✔ {cancer_type} saved: {subset_df.shape}")

Number of CpG sites remaining after dropping rows with NaN: 296018
   cg00000108  cg00000236  cg00000292  cg00000321  cg00000363  cg00000622  \
0    0.953630    0.912979    0.765790    0.387424    0.254435    0.016053   
1    0.972363    0.923796    0.746361    0.419409    0.519763    0.014923   
2    0.956964    0.888815    0.698911    0.268164    0.180543    0.019389   
3    0.966731    0.947110    0.761455    0.462856    0.443001    0.020803   
4    0.969658    0.933957    0.633932    0.372154    0.155002    0.015081   

   cg00000658  cg00000714  cg00000721  cg00000734  ...  ctl_67735372  \
0    0.908794    0.119768    0.954978    0.093893  ...      0.049045   
1    0.910423    0.157720    0.950980    0.069575  ...      0.060875   
2    0.884833    0.146261    0.931858    0.097583  ...      0.060984   
3    0.887060    0.119726    0.942514    0.080016  ...      0.049134   
4    0.877770    0.109926    0.950364    0.078565  ...      0.052831   

   ctl_67775306  ctl_68641317  ctl_68

In [ ]:
import pandas as pd
import os

input_files = {
    "COAD": 'filtered_methylation_data/coad_top300kMAD_cpg.parquet',
    "READ": 'filtered_methylation_data/read_top300kMAD_cpg.parquet',
    "LUAD": 'filtered_methylation_data/luad_top300kMAD_cpg.parquet'
}

dfs = {cancer_type: pd.read_parquet(file) for cancer_type, file in input_files.items()}

expected_num_sites = 250000
for cancer_type, df in dfs.items():
    if df.shape[1] != expected_num_sites:
        raise ValueError(f"ERROR: {cancer_type} dataset has {df.shape[1]} CpG sites instead of {expected_num_sites}. You need to start over.")

reference_cpg_sites = list(dfs["COAD"].columns)

for cancer_type, df in dfs.items():
    if list(df.columns) != reference_cpg_sites:
        raise ValueError(f"ERROR: {cancer_type} CpG sites do NOT match exactly. You need to start over.")
